## Exploring the CAJ AER paper.

In [5]:
import json, glob
from caj_tools import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Get all the graphs -- they're all set.

In [2]:
graphs = {}
for f in glob.glob("f*/*.json"):
    with open(f) as i:
        graphs[f.replace(".json", "")] = json.load(i)

This is what the graphs look like.

In [3]:
graphs["f5/a"]

[{'id': 1, 'nei': [2, 5, 8]},
 {'id': 2, 'nei': [1, 3, 6]},
 {'id': 3, 'nei': [2, 4, 7]},
 {'id': 4, 'nei': [3, 5, 8]},
 {'id': 5, 'nei': [1, 4, 6]},
 {'id': 6, 'nei': [2, 5, 7]},
 {'id': 7, 'nei': [3, 6, 8]},
 {'id': 8, 'nei': [4, 7, 1]}]

### In Figure 1, agent 1 benefits from agent 3 in the long run (s₁(a) > s₁(b)).  But for s<sub>t-1</sub> = (0, 1, 0) as given, they are anticorrelated.

In [8]:
iterate(graphs["f1/a"], N = 20000000)
iterate(graphs["f1/b"], N = 20000000)

In [9]:
unemployment(graphs["f1/a"][0]["hist"])
unemployment(graphs["f1/b"][0]["hist"])

0.07870845000000004

0.07842784999999997

In [10]:
s1_hist = graphs["f1/a"][0]["hist"]
s2_hist = graphs["f1/a"][1]["hist"]
s3_hist = graphs["f1/a"][2]["hist"]

s1_cond = [x for xi, x in enumerate(graphs["f1/a"][0]["hist"])
           if xi and s1_hist[xi-1] == 0 and s2_hist[xi-1] == 1 and s3_hist[xi-1] == 0]

s3_cond = [x for xi, x in enumerate(graphs["f1/a"][2]["hist"])
           if xi and s1_hist[xi-1] == 0 and s2_hist[xi-1] == 1 and s3_hist[xi-1] == 0]

correlation(s1_cond, s3_cond)

-0.013624084186525979

### In Figure 2 A-D, the employment rates are 0.132, 0.083, 0.063, and 0.050.

In [11]:
iterate(graphs["f2/a"], N = 10000000)
iterate(graphs["f2/b"], N = 10000000)
iterate(graphs["f2/c"], N = 10000000)
iterate(graphs["f2/d"], N = 10000000)

In [12]:
unemployment(graphs["f2/a"][0]["hist"])
unemployment(graphs["f2/b"][0]["hist"])
unemployment(graphs["f2/c"][0]["hist"])
unemployment(graphs["f2/d"][0]["hist"])

0.1324147

0.07893640000000002

0.05666009999999999

0.0464232

No, but their footnotes 8 and 9 are not encouraging:

_The numbers for more than one agent are obtained from simulations in Maple􏰥. We simulate the economy over a large number of periods (hundreds of thousands) and calculate observed unemployment averages and correlations. The programs are available upon request from the authors. The correlation numbers are only moderately accurate, even after several hundred thousand periods._

_In some cases, the correlations are indistinguishable to the accuracy of our simulations._

If I use as few iterations, I can reproduce their numbers.

### The Corr(s₁ s₂) are --, 0.041, 0.025, and 0.025.

In [13]:
correlation(graphs["f2/a"][0]["hist"], graphs["f2/a"][1]["hist"])
correlation(graphs["f2/b"][0]["hist"], graphs["f2/b"][1]["hist"])
correlation(graphs["f2/c"][0]["hist"], graphs["f2/c"][1]["hist"])
correlation(graphs["f2/d"][0]["hist"], graphs["f2/d"][1]["hist"])

0.0012435095464074816

0.051988558283715895

0.02077987511936461

0.029593582870619525

### Figure 4: The employment rates for types 1, 2, and 3 are 0.047, 0.048, and 0.050.

In [14]:
iterate(graphs["f4/a"], N = 10000000)

unemp_rates = {}
for node in range(10):
    unemp_rates[node+1] = unemployment(graphs["f4/a"][node]["hist"])
    print("{:d} :: {:.4f}".format(node+1, unemp_rates[node+1]))

(unemp_rates[1] + unemp_rates[6])/2
(unemp_rates[2] + unemp_rates[5] + unemp_rates[7] + unemp_rates[10])/4
(unemp_rates[3] + unemp_rates[4] + unemp_rates[8] + unemp_rates[9]) /4

1 :: 0.0454
2 :: 0.0458
3 :: 0.0457
4 :: 0.0459
5 :: 0.0457
6 :: 0.0449
7 :: 0.0459
8 :: 0.0460
9 :: 0.0461
10 :: 0.0456


0.04518354999999996

0.045746449999999966

0.045919975

### The average path length matters for unemployment

We can get the path lengths right, but the employment rates are slightly lower than they quote, and only very slightly different.

In [15]:
avg_path_length(graphs["f5/a"]), avg_path_length(graphs["f5/b"])

(1.5714285714285714, 1.7857142857142856)

In [16]:
iterate(graphs["f5/a"], N = 5000000)
iterate(graphs["f5/b"], N = 5000000)

sum([unemployment(graphs["f5/a"][n]["hist"]) for n in range(8)])/8
sum([unemployment(graphs["f5/b"][n]["hist"]) for n in range(8)])/8

0.04539747500000001

0.04580232499999998

### §III, Fig. 6: for connected agents, expectations of next-round employment fall with length of unemployment duration, because duration is indicative of peers current status.

In [17]:
def conditional_prob_periods(hist, periods):
    
    cond = []
    for xi in range(periods, len(hist)):
        if not any([hist[xpi] for xpi in range(xi-periods, xi)]):
            cond.append(hist[xi])
            
    return sum(cond)/len(cond)

conditional_prob_periods(graphs["f2/b"][0]["hist"], 1)
conditional_prob_periods(graphs["f2/b"][0]["hist"], 2)
conditional_prob_periods(graphs["f2/b"][0]["hist"], 10)
conditional_prob_periods(graphs["f2/b"][0]["hist"], 20)

conditional_prob_periods(graphs["f2/d"][0]["hist"], 1)
conditional_prob_periods(graphs["f2/d"][0]["hist"], 2)
conditional_prob_periods(graphs["f2/d"][0]["hist"], 10)
conditional_prob_periods(graphs["f2/d"][0]["hist"], 20)

0.17536903127074455

0.17407294748776372

0.1667168447990366

0.1675246856274791

0.30654069516965654

0.30072439007722274

0.27892108824490175

0.253475935828877